# INSTALL: packages

In [ ]:
pip install ipynb

In [ ]:
pip install graphviz

In [1]:
import os
import pickle
from ipynb.fs.full.new-WalletClustering_MIH_fast import iterMultiInputClustering
from graphviz import Digraph

# EXECUTE: WalletClustering_neo4jConnect notebook

In [2]:
%run ./WalletClustering_neo4jConnect.ipynb # includes Neo4J connector
# methods & variables of notebook can be referenced

You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


# DEFINE: Make graph of flows

## create blacklist of associations

In [10]:
def createAssocBlacklist():
    # use existing assocBlacklist if exists
    if not os.path.exists('output\\assocBlacklist.pickle'):
        assocBlacklist = []
        dirname = os.path.dirname(os.path.realpath('__file__'))
  
        # iterate scraped blacklist folders to get blacklisted associations
        if os.path.isdir('..\\WalletExplorerScraper\\Output\\exchange\\'):
            outputScrapeExchangeFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\exchange\\')
            for subfolder in os.listdir(outputScrapeExchangeFolder):
                assocBlacklist.append(subfolder.split("_",1)[1])
                continue
          
        if os.path.isdir('..\\WalletExplorerScraper\\Output\\mixer\\'):
            outputScrapeMixerFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\mixer\\')
            for subfolder in os.listdir(outputScrapeMixerFolder):
                assocBlacklist.append(subfolder.split("_",1)[1])
                continue

        if os.path.isdir('..\\WalletExplorerScraper\\Output\\giant_wallet\\'):
            outputScrapeGiantWalletFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\giant_wallet\\')
            for subfolder in os.listdir(outputScrapeGiantWalletFolder):
                assocBlacklist.append(subfolder.split("_",1)[1])
                continue

        with open('output\\assocBlacklist.pickle', 'wb') as export:
            pickle.dump(set(assocBlacklist), export)

    assocBlacklist = pickle.load(open('output\\assocBlacklist.pickle', 'rb'))
    return assocBlacklist

## check for blacklisted association

In [ ]:
#check if association is part of blacklisted associations
def assocBlacklistCheck(association, assocBlacklist):
    if association in assocBlacklist:
        return True
    else:
        return False

## query for input address association

In [ ]:
def getAssoc(address):
    # return association of input address
    assocTemplate = '''
    MATCH (a:Address {address: "%s"})
    RETURN a.association
    '''

    association = conn.query(assocTemplate % address, db='neo4j')
    
    if association is None:
        # run MI heuristic on address and query again
        iterMultiInputClustering(address) #should also flag new association in neo4J
        association = conn.query(assocTemplate % address, db='neo4j')
        return association

    else:
        assocBlacklist = createAssocBlacklist()
        if assocBlacklistCheck(association, assocBlacklist):
            #throw exception in case association is blacklisted
            raise Exception('Blacklist')
        else:
            return association   

## gather input and output addresses to association

In [ ]:
#get all transactions that are either output or input to an association (user)
def getUserTransactions(association):
    userOutputTransactionsTemplate = '''
    MATCH (u:Address{association:"%s"})-[:SENDS]->(t:Transaction)-[:SENDS]->(o:Address),
    (u:Address)-[:SENDS]->(t:Transaction)-[:SENDS]->(o:Address)
    RETURN DISTINCT o
    '''
    outputResponse = conn.query(userOutputTransactionsTemplate % association, db='neo4j')
    userOutput = [oR[0] for oR in outputResponse]

    userInputTransactionsTemplate = '''
    MATCH (u:Address{association:"%s"})<-[:RECEIVES]-(t:Transaction)<-[:RECEIVES]->(i:Address),
    (u:Address)<-[:RECEIVES]-(t:Transaction)<-[:RECEIVES]-(i:Address)
    RETURN DISTINCT i
    '''
    inputResponse = conn.query(userInputTransactionsTemplate % association, db='neo4j')
    userInput = [iR[0] for iR in inputResponse]

    return userOutput, userInput

## query for input and output address asociations

In [ ]:
def buildFlowGraph(inputAddress):
    try:
        association = getAssoc(inputAddress)
    except:
        raise Exception('FU') # adjust to use case

    outputs, inputs = getUserTransactions(association)
    
    outputAssocs = []
    for o in outputs:
        outputAssocs.append(getAssoc(o))
    outputAssocs = set(outputAssocs)
    
    inputAssocs = []
    for i in inputs:
        inputAssocs.append(getAssoc(i))
    inputAssocs = set(inputAssocs)

    # build graph (adjust to use case)
    dot = Digraph()
    dot.node(association, association)
        
    for outNode in outputAssocs:
        dot.note(outNode, outNode)
        dot.edge(association, outNode)

    for inNode in inputAssocs:
        dot.note(inNode, inNode)
        dot.edge(inNode, association)
        
    return dot

# RUN: Make graph of flows

In [11]:
inputAddress = ''

{'helix', 'bittrex.com', 'huobi.com'}


In [16]:
graph = buildFlowGraph(inputAddress)
graph.render(view=True)

True
